In [1]:
from pynq import DefaultHierarchy
from pynq import allocate
from math import log
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import time

# DMA Driver Class

In [2]:
class FFT_Block_Driver(DefaultHierarchy):
    def __init__(self, description):
        super().__init__(description)
        self.configuration = 0
        self.fft_size = 0

    def _convert_to_data(self, fft_direction, size):
        fft_direction.zfill(8)
        byte2 = '0'*8
        x = int(log(size, 2))
        fft_size = bin(x)[2:]
        fft_size.zfill(8)
        tdata = fft_direction + byte2 + fft_size
        return int(tdata, 2)

    def configure(self, fft_direction, fft_size):
        self.configuration = self._convert_to_data(fft_direction, fft_size)
        conf_buffer = allocate(1, np.uint32)
        conf_buffer[0] = self.configuration
        
        st = time.time()
        self.config_dma.sendchannel.transfer(conf_buffer)
        self.config_dma.sendchannel.wait()
        et = time.time()
        
        del conf_buffer
        
        return (et-st)

    def stream_fft(self, input_buffer):
        output_buffer = allocate(samples.value, np.csingle)
        
        st = time.time()
        self.data_dma.sendchannel.transfer(input_buffer)
        self.data_dma.recvchannel.transfer(output_buffer)
        self.data_dma.sendchannel.wait()
        self.data_dma.recvchannel.wait()
        et = time.time()

        return (output_buffer, (et-st))

    @staticmethod
    def checkhierarchy(description):
        if 'data_dma' in description['ip'] and 'config_dma' in description['ip']:
                return True
        return False

# Set Up

In [3]:
from pynq import Overlay
ol = Overlay("reconfig_fft.bit")

In [4]:
# ol?

In [5]:
fft = ol.fft_block

# Create Data

In [6]:
f1 = widgets.FloatSlider(min = 10, max = 50, step = 1, description = "f1", value=10)
f2 = widgets.FloatSlider(min = 10, max = 50, step = 1, description = "f2", value=30)
f3 = widgets.FloatSlider(min = 10, max = 50, step = 1, description = "f3", value=50)
A1 = widgets.FloatSlider(min = 100, max = 200, step = 1, description = "A1", value=100)
A2 = widgets.FloatSlider(min = 100, max = 200, step = 1, description = "A2", value=150)
A3 = widgets.FloatSlider(min = 100, max = 200, step = 1, description = "A3", value=200)
c1 = widgets.Checkbox(value=True, description="PS Implementation")
c2 = widgets.Checkbox(value=True, description="PL Implementation")
samples = widgets.Dropdown(options=[64, 128, 256, 512, 1024, 2048, 4096, 8192], value=8192, description='samples')
T = 1

In [7]:
def create_data(A1, A2, A3, f1, f2, f3, samples):
    t = np.linspace(0, T, samples)
    data = A1*np.sin(2*np.pi*f1*t) + A2*np.sin(2*np.pi*f2*t) + A3*np.sin(2*np.pi*f3*t)
    return data, t

def plot_graph(data, t):
    plt.plot(t, np.abs(data), label = "Input Data")
    plt.xlabel('Time (s)')
    plt.ylabel('Absolute Value')
    plt.legend()
    plt.show()

# FFT function with options for PS PL Implementation

In [8]:
def plot_fft(A1, A2, A3, f1, f2, f3, samples, ps, pl):
    data, t = create_data(A1, A2, A3, f1, f2, f3, samples)
    plot_graph(data, t)
    freq = np.fft.fftshift(np.fft.fftfreq(samples, T/samples))
    
    if ps:
        st = time.time()
        ps_output = np.abs(np.fft.fft(data, samples)) / samples
        et = time.time()
        
        print("PS Execution Time: " + str(et - st))
        
        ps_output_shifted = np.fft.fftshift(ps_output)
        plt.plot(freq, ps_output_shifted, label='PS Output')
        
    if pl:       
        input_buffer = allocate(samples, np.csingle)
        np.copyto(input_buffer, data)
        
        config_time = fft.configure('1', samples)
        output_buffer, fft_time = fft.stream_fft(input_buffer)
        
        print("PL Execution Time Config: " + str(config_time))
        print("PL Execution Time FFT: " + str(fft_time))
        
        pl_output_shifted = np.fft.fftshift(np.abs(output_buffer) / samples)
        plt.plot(freq, pl_output_shifted, label='PL Output')
        
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')
    plt.legend()
    plt.show()
        
    if ps and pl:
        plt.figure()
        plt.plot(freq, np.abs(ps_output_shifted - pl_output_shifted), label='Difference')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Magnitude')
        plt.legend()
        plt.show()

In [9]:
widgets.interact_manual(plot_fft, A1=A1, A2=A2, A3=A3, f1=f1, f2=f2, f3=f3, samples=samples, ps=c1, pl=c2)

interactive(children=(FloatSlider(value=100.0, description='A1', max=200.0, min=100.0, step=1.0), FloatSlider(…

<function __main__.plot_fft>